In [1]:
"""
Weather Data Recorder
A complete Python application for recording, managing, and analyzing weather data.

Features:
- Add weather data with date validation
- Prevent duplicate date entries
- View all recorded data
- Summarize weather trends using Pandas
- Export data to CSV format
- Interactive command-line interface

Requirements:
- pandas library: pip install pandas
"""

import pandas as pd
from datetime import datetime
import os
import sys


class WeatherDataRecorder:
    """Main class for handling weather data operations."""

    def __init__(self):
        """Initialize the weather data recorder."""
        self.weather_data = []  # List of dictionaries to store weather records
        self.recorded_dates = set()  # Set to track unique dates and prevent duplicates

    def is_valid_date(self, date_str):
        """
        Validate if the given date string is in correct format (YYYY-MM-DD).

        Args:
            date_str (str): Date string to validate

        Returns:
            bool: True if valid, False otherwise
        """
        try:
            datetime.strptime(date_str, "%Y-%m-%d")
            return True
        except ValueError:
            return False

    def add_weather_data(self):
        """
        Add new weather data entry with validation.
        Prevents duplicate dates and validates input data.
        """
        print("\n=== Add Weather Data ===")

        # Get and validate date
        date = input("Enter date (YYYY-MM-DD): ").strip()
        if not self.is_valid_date(date):
            print("❌ Invalid date format! Please use YYYY-MM-DD format.")
            return

        if date in self.recorded_dates:
            print("❌ Date already exists. Cannot add duplicate entry.")
            return

        # Get and validate temperature
        try:
            temp = float(input("Enter temperature (°C): "))
        except ValueError:
            print("❌ Invalid temperature! Please enter a numeric value.")
            return

        # Get weather condition
        condition = input("Enter weather condition (e.g., Sunny, Rainy, Cloudy): ").strip()
        if not condition:
            print("❌ Weather condition cannot be empty!")
            return

        # Add the record
        weather_record = {
            "Date": date,
            "Temperature": temp,
            "Condition": condition.title()  # Capitalize first letter
        }

        self.weather_data.append(weather_record)
        self.recorded_dates.add(date)
        print("✅ Weather data added successfully!")
        print(f"Record: {date} | {temp}°C | {condition.title()}")

    def view_weather_data(self):
        """Display all recorded weather data in a formatted table."""
        print("\n=== All Weather Data ===")

        if not self.weather_data:
            print("📭 No data recorded yet.")
            return

        # Sort data by date for better readability
        sorted_data = sorted(self.weather_data, key=lambda x: x["Date"])

        print(f"{'Date':<12} {'Temperature':<12} {'Condition':<15}")
        print("-" * 40)

        for entry in sorted_data:
            print(f"{entry['Date']:<12} {entry['Temperature']:<12}°C {entry['Condition']:<15}")

        print(f"\nTotal records: {len(self.weather_data)}")

    def summarize_data(self):
        """
        Analyze and summarize weather data trends using Pandas.
        Shows statistics like average, max, min temperatures and condition counts.
        """
        print("\n=== Weather Data Summary ===")

        if not self.weather_data:
            print("📭 No data to summarize.")
            return

        try:
            # Create DataFrame for analysis
            df = pd.DataFrame(self.weather_data)

            # Temperature statistics
            print("🌡️  Temperature Statistics:")
            print(f"   Average Temperature: {df['Temperature'].mean():.2f}°C")
            print(f"   Maximum Temperature: {df['Temperature'].max():.2f}°C")
            print(f"   Minimum Temperature: {df['Temperature'].min():.2f}°C")
            print(f"   Temperature Range: {df['Temperature'].max() - df['Temperature'].min():.2f}°C")

            # Weather condition analysis
            print("\n🌤️  Weather Condition Analysis:")
            condition_counts = df["Condition"].value_counts()
            for condition, count in condition_counts.items():
                percentage = (count / len(df)) * 100
                print(f"   {condition}: {count} days ({percentage:.1f}%)")

            # Date range
            print(f"\n📅 Data Range:")
            print(f"   From: {min(df['Date'])}")
            print(f"   To: {max(df['Date'])}")
            print(f"   Total days recorded: {len(df)}")

        except Exception as e:
            print(f"❌ Error during analysis: {e}")

    def export_to_csv(self):
        """Export weather data to CSV file."""
        print("\n=== Export to CSV ===")

        if not self.weather_data:
            print("📭 No data to export.")
            return

        try:
            # Create DataFrame and sort by date
            df = pd.DataFrame(self.weather_data)
            df = df.sort_values('Date')

            # Get filename from user or use default
            filename = input("Enter filename (or press Enter for 'weather_data.csv'): ").strip()
            if not filename:
                filename = "weather_data.csv"
            elif not filename.endswith('.csv'):
                filename += '.csv'

            # Export to CSV
            df.to_csv(filename, index=False)
            print(f"✅ Data exported successfully to '{filename}'")
            print(f"📁 File location: {os.path.abspath(filename)}")

        except Exception as e:
            print(f"❌ Error exporting data: {e}")

    def import_from_csv(self):
        """Import weather data from CSV file."""
        print("\n=== Import from CSV ===")

        filename = input("Enter CSV filename: ").strip()
        if not filename:
            print("❌ Filename cannot be empty!")
            return

        try:
            if not os.path.exists(filename):
                print(f"❌ File '{filename}' not found!")
                return

            # Read CSV file
            df = pd.read_csv(filename)

            # Validate required columns
            required_columns = ['Date', 'Temperature', 'Condition']
            if not all(col in df.columns for col in required_columns):
                print(f"❌ CSV must contain columns: {required_columns}")
                return

            # Import data
            imported_count = 0
            skipped_count = 0

            for _, row in df.iterrows():
                date = str(row['Date'])

                # Skip if date already exists
                if date in self.recorded_dates:
                    skipped_count += 1
                    continue

                # Validate date format
                if not self.is_valid_date(date):
                    print(f"⚠️  Skipping invalid date: {date}")
                    skipped_count += 1
                    continue

                # Add record
                self.weather_data.append({
                    "Date": date,
                    "Temperature": float(row['Temperature']),
                    "Condition": str(row['Condition']).title()
                })
                self.recorded_dates.add(date)
                imported_count += 1

            print(f"✅ Import completed!")
            print(f"   Imported: {imported_count} records")
            print(f"   Skipped: {skipped_count} records")

        except Exception as e:
            print(f"❌ Error importing data: {e}")

    def clear_all_data(self):
        """Clear all recorded weather data with confirmation."""
        print("\n=== Clear All Data ===")

        if not self.weather_data:
            print("📭 No data to clear.")
            return

        confirmation = input(f"⚠️  Are you sure you want to delete all {len(self.weather_data)} records? (yes/no): ").strip().lower()

        if confirmation in ['yes', 'y']:
            self.weather_data.clear()
            self.recorded_dates.clear()
            print("✅ All data cleared successfully!")
        else:
            print("❌ Operation cancelled.")

    def display_menu(self):
        """Display the main menu options."""
        print("\n" + "="*50)
        print("🌤️  WEATHER DATA RECORDER")
        print("="*50)
        print("1. ➕ Add Weather Data")
        print("2. 👁️  View All Data")
        print("3. 📊 Summarize Data")
        print("4. 💾 Export to CSV")
        print("5. 📁 Import from CSV")
        print("6. 🗑️  Clear All Data")
        print("7. ❌ Exit")
        print("="*50)

    def run(self):
        """Main program loop."""
        print("🌤️  Welcome to Weather Data Recorder!")
        print("Track and analyze your weather data efficiently.")

        while True:
            self.display_menu()
            choice = input("Choose an option (1-7): ").strip()

            if choice == "1":
                self.add_weather_data()
            elif choice == "2":
                self.view_weather_data()
            elif choice == "3":
                self.summarize_data()
            elif choice == "4":
                self.export_to_csv()
            elif choice == "5":
                self.import_from_csv()
            elif choice == "6":
                self.clear_all_data()
            elif choice == "7":
                print("\n👋 Thank you for using Weather Data Recorder!")
                print("🌟 Your data has been saved in memory during this session.")
                break
            else:
                print("❌ Invalid choice. Please select a number between 1-7.")

            # Pause for user to read output
            input("\nPress Enter to continue...")


def check_dependencies():
    """Check if required dependencies are installed."""
    try:
        import pandas
        return True
    except ImportError:
        print("❌ Error: pandas library is required but not installed.")
        print("💡 Please install it using: pip install pandas")
        return False


def main():
    """Main function to run the Weather Data Recorder application."""
    # Check dependencies
    if not check_dependencies():
        sys.exit(1)

    # Create and run the application
    recorder = WeatherDataRecorder()

    try:
        recorder.run()
    except KeyboardInterrupt:
        print("\n\n👋 Program interrupted by user. Goodbye!")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")
        print("Please try again or contact support.")


if __name__ == "__main__":
    main()

🌤️  Welcome to Weather Data Recorder!
Track and analyze your weather data efficiently.

🌤️  WEATHER DATA RECORDER
1. ➕ Add Weather Data
2. 👁️  View All Data
3. 📊 Summarize Data
4. 💾 Export to CSV
5. 📁 Import from CSV
6. 🗑️  Clear All Data
7. ❌ Exit
Choose an option (1-7): 1

=== Add Weather Data ===
Enter date (YYYY-MM-DD): 2024-1-1
Enter temperature (°C): 16
Enter weather condition (e.g., Sunny, Rainy, Cloudy): Cloudy
✅ Weather data added successfully!
Record: 2024-1-1 | 16.0°C | Cloudy

Press Enter to continue...

🌤️  WEATHER DATA RECORDER
1. ➕ Add Weather Data
2. 👁️  View All Data
3. 📊 Summarize Data
4. 💾 Export to CSV
5. 📁 Import from CSV
6. 🗑️  Clear All Data
7. ❌ Exit
Choose an option (1-7): 1

=== Add Weather Data ===
Enter date (YYYY-MM-DD): 2024-4-4
Enter temperature (°C): 25
Enter weather condition (e.g., Sunny, Rainy, Cloudy): Sunny
✅ Weather data added successfully!
Record: 2024-4-4 | 25.0°C | Sunny

Press Enter to continue...

🌤️  WEATHER DATA RECORDER
1. ➕ Add Weather Dat